In [1]:
import os 
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI 
from langchain_core.output_parsers import StrOutputParser 
from langchain.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [3]:
model.invoke("Erkläre LLMs in drei Sätzen")
# returns JSON data with model output and metadata 

AIMessage(content='Ein Master of Laws (LLM) ist ein postgraduales Studium für Juristen, die bereits einen Abschluss in Jura haben. Es ermöglicht den Studierenden, sich auf ein spezifisches Rechtsgebiet zu spezialisieren oder ihre Kenntnisse in einem bestimmten Bereich zu vertiefen. Ein LLM kann in verschiedenen Ländern angeboten werden und wird oft von internationalen Studierenden gewählt, um ihr Wissen im internationalen Recht zu erweitern.', response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 18, 'total_tokens': 126}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a8d842e-c280-4adc-bdbb-c8fb5f5e2b61-0')

In [4]:
response = model.invoke("Erkläre LLMs in drei Sätzen")
response.content

'Ein LLM (Master of Laws) ist ein postgraduales Studium für Juristen, um ihre Kenntnisse auf einem bestimmten Rechtsgebiet zu vertiefen. LLM-Programme werden in vielen Ländern weltweit angeboten und können sowohl allgemeine Rechtspraxis als auch spezialisierte Bereiche abdecken. Absolventen eines LLM-Programms haben in der Regel bessere Karrierechancen und können in verschiedenen Bereichen des Rechts tätig werden.'

In [5]:
parser = StrOutputParser() # Initialisierung eines Objekts zum Extrahieren des Strings 
chain = model | parser # Output des Model wird in parser Objekt gepiped
chain.invoke("Erkläre LLMs in drei Sätzen")

'LLMs, oder Language Model Models, sind eine Art von künstlicher Intelligenz, die dazu entwickelt wurden, natürlich-sprachliche Daten zu verstehen und zu generieren. Sie werden häufig für Aufgaben wie Übersetzung, Textgenerierung und Textklassifizierung verwendet. LLMs basieren oft auf neuronalen Netzwerken und werden durch umfangreiche Trainingsdatensätze aufgebaut, um eine hohe Genauigkeit und Leistung zu erzielen.'

In [6]:
template = """
Beantworte die Frage basierend auf dem unten gegebenen Kontext. 
Wenn das Beantworten der Frage nicht möglich ist durch den gegebenen Kontekt, antworte "Ich weiß es nicht". 

Kontext: {context}

Frage: {question}
"""

prompt = ChatPromptTemplate.from_template(template) # Template Prompt Objekt erzeugen


In [7]:
model.invoke(prompt.format(context="Es gibt auschließlich weiße Schwäne.", question = "Gibt es schwarze Schwäne?")).content


'Ich weiß es nicht.'

In [8]:
chain = prompt | model | parser
chain.invoke({
    "context": "Es gibt ausschließlich weiße Schwäne.", 
    "question": "Gibt es schwarze Schwäne?"
})

'Ich weiß es nicht.'

In [66]:
from operator import itemgetter

diff_prompt = ChatPromptTemplate.from_template(
    "Übersetze {answer} in {language}"
)

diff_chain = (
    # dic um Input zu definieren, dies wird dann in diff_prompt gepiped
    {"answer": chain, # Antwort wird aus der anderen Chain als Input genommen
     "language": itemgetter("language")} # Sprache aus Dictonary
    | diff_prompt | model | parser 
)

diff_chain.invoke(
    {
        "context": "Alle Schwäne sind weiß, außer in Australien. Dort gibt es schwarze.",
        "question": "Gibt es schwarze Schwäne?", 
        "language": "Italienisch"
    }
)

'Sì, in Australia ci sono cigni neri.'

In [4]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Gibt es schwarze Schwäne?")

print(f"Embedding length: {len(embedded_query)}") # Dimension der Embedding Liste

Embedding length: 1536
[0.00901121263534296, -0.03669295798076368, -0.00873078020641617, -0.025039455366582045, -0.01990443465229751, 0.006879930087054519, -0.0007376917735520665, -0.012351468412199479, -0.009098458010849193, -0.04162856202435205]


In [6]:
sen1 = embeddings.embed_query("Es gibt in Europa nur weiße Schwäne")
sen2 = embeddings.embed_query("Es gibt schwarze Schwäne in Australien")

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

query_sen1_sim = cosine_similarity([embedded_query], [sen1])[0][0]
query_sen2_sim = cosine_similarity([embedded_query], [sen2])[0][0]
query_sen1_sim, query_sen2_sim

(0.8740940461805541, 0.9229002354973087)